In [1]:
#Import dependencies
import pandas as pd
import numpy as np
import 

In [ ]:
# Narrative/intro goes here

In [2]:
# Stacy start

In [ ]:
# Plot the naturalization data

## CLEAN THIS UP TO MATCH CORRECT VARIABLES

plt.plot(naturalization.index, naturalization[['Petitions filed', 'Naturalized, total', 'Petitions denied']])
plt.tight_layout()

# Spike in 1996 - and general increase - can be explained by this bill: https://www.vox.com/2016/4/28/11515132/iirira-clinton-immigration
# Need to plot just the 2000 - 2010, then 2015 - 2018 data - to examine whether there are trends

In [3]:
# Stacy end

In [4]:
# Kana start

In [5]:
# Kana end

In [6]:
# Satish start

In [7]:
# Satish end

In [1]:
# Umar start

In [ ]:
# Umar End

In [ ]:
# Narrative/summary goes here